In [1]:
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
sys.path.append('..')

import tensorflow as tf

from src.data.cifar100 import CIFAR_100_CLASSES, load_cifar100_data
from src.meta.tensor_apis import TensorApi
from src.meta.meta import MetaModel, MetaModelFactory
from src.utils.debug_utils import print_list
from src.utils.logger import HuliLogging

print(tf.__version__)

/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

1.14.0


/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([

In [2]:
logger = HuliLogging.get_logger(__name__)
HuliLogging.attach_stdout()

# Load data

In [3]:
%%bash

find ../models -type f | sort | grep conv

../models/conv_001/conv_001_checkpoint.h5
../models/conv_001/conv_001.h5
../models/conv_001/conv_001_int8.tflite
../models/conv_001/conv_001_no_optimizer.h5
../models/conv_001/conv_001_weights.h5
../models/conv_002/conv_002_checkpoint.h5
../models/conv_002/conv_002.h5
../models/conv_002/conv_002_no_optimizer.h5
../models/conv_002/conv_002.pb
../models/conv_003/conv_003_checkpoint.h5
../models/conv_003/conv_003.h5
../models/conv_003/conv_003_int8.tflite
../models/conv_003/conv_003_no_optimizer.h5
../models/conv_003/conv_003.pb
../models/conv_003/conv_003_weights.h5
../models/conv_004/conv_004_checkpoint.h5
../models/conv_004/conv_004.h5
../models/conv_004/conv_004_no_optimizer.h5
../models/conv_004/conv_004.pb
../models/conv_005/conv_005_checkpoint.h5
../models/conv_005/conv_005.h5
../models/conv_005/conv_005_no_optimizer.h5
../models/conv_005/conv_005.pb
../models/conv_006/conv_006_checkpoint.h5
../models/conv_006/conv_006.h5
../models/conv_006/conv_006_no_optimizer.h5
../models/conv_0

In [4]:
_NAME = 'conv'
_EPOCH = 1
_ADDITIONAL_EPOCHS = 10
_DO_FIT = True

In [5]:
(train_images, train_labels), (test_images, test_labels) = load_cifar100_data()

# Init as `KERAS` mode

In [6]:
ret, model = MetaModelFactory.from_h5(_NAME, _EPOCH)
m = model
print(type(m))

2019-08-08 17:36:44,979 DEBUG [src.meta.keras:276] (MainThread) conv Loading keras model from /space/code/deep-learning/models/conv_001/conv_001.h5...


W0808 17:36:45.001389 140100827197568 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0808 17:36:45.002273 140100827197568 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0808 17:36:45.003095 140100827197568 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6

<class 'src.meta.meta.MetaModel'>


In [7]:
m.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

0

In [8]:
m.epoch

1

In [9]:
m.mode

<TensorApi.KERAS: 'keras'>

In [10]:
if _DO_FIT:
    for i in range(m.epoch, m.epoch+_ADDITIONAL_EPOCHS):
        model.fit(train_images, train_labels, epochs=i+1, initial_epoch=i)
        model.freeze_session()

2019-08-08 17:36:46,507 DEBUG [src.meta.keras:63] (MainThread) conv keras tensorboard callback attached. Visualize by running: 
2019-08-08 17:36:46,508 DEBUG [src.meta.keras:64] (MainThread) > tensorboard --logdir=/space/code/deep-learning/tmp/tensorboard/20190808-173646_conv
2019-08-08 17:36:46,509 DEBUG [src.meta.keras:66] (MainThread) conv keras checkpoint callback attached. Logging to /space/code/deep-learning/models/conv_{epoch:03d}/conv_{epoch:03d}_checkpoint.h5...
2019-08-08 17:36:46,509 INFO  [src.meta.keras:73] (MainThread) Creating /space/code/deep-learning/models/conv_002 (if not already exists)...
Epoch 2/2
50000/50000 [==============================] - 10s 193us/sample - loss: 3.0967 - acc: 0.2534
2019-08-08 17:36:56,781 WARN  [src.meta.keras:190] (MainThread) conv Bug Workaround: Saving h5 file to ensure it can be loaded...
2019-08-08 17:36:56,806 WARN  [src.meta.keras:193] (MainThread) conv Bug Workaround: Saving h5 file without optimizer to ensure it can be loaded...
20

W0808 17:36:57.038231 140100827197568 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.
W0808 17:36:57.039471 140100827197568 deprecation_wrapper.py:119] From ../src/meta/keras.py:213: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

W0808 17:36:57.040187 140100827197568 deprecation_wrapper.py:119] From ../src/meta/keras.py:216: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



2019-08-08 17:36:57,040 INFO  [src.meta.keras:217] (MainThread) conv freeze_var_names = ['conv2d/bias', 'conv2d_1/kernel', 'batch_normalization_2/beta', 'fc100/bias', 'batch_normalization_1/beta', 'conv2d_2/kernel', 'batch_normalization/moving_variance', 'batch_normalization_2/gamma', 'batch_normalization/moving_mean', 'batch_normalization_1/moving_variance', 'conv2d_1/bias', 'batch_normalization_2/moving_variance', 'batch_normalization/beta', 'batch_normalization_1/moving_mean', 'batch_normalization_2/moving_mean', 'fc100/kernel', 'batch_normalization_1/gamma', 'conv2d_2/bias', 'batch_normalization/gamma', 'conv2d/kernel']
2019-08-08 17:36:57,042 INFO  [src.meta.keras:221] (MainThread) conv output_names = ['fc100/Softmax', 'conv2d/kernel', 'conv2d/bias', 'batch_normalization/gamma', 'batch_normalization/beta', 'batch_normalization/moving_mean', 'batch_normalization/moving_variance', 'conv2d_1/kernel', 'conv2d_1/bias', 'batch_normalization_1/gamma', 'batch_normalization_1/beta', 'batch

W0808 17:36:57.045196 140100827197568 deprecation.py:323] From ../src/meta/keras.py:231: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0808 17:36:57.045760 140100827197568 deprecation.py:323] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


2019-08-08 17:36:57,068 INFO  [src.meta.keras:236] (MainThread) conv Saving frozen graph to /space/code/deep-learning/models/conv_002/conv_002.pb...


...

2019-08-08 17:37:55,660 INFO  [src.meta.keras:217] (MainThread) conv freeze_var_names = ['conv2d/bias', 'conv2d_1/kernel', 'batch_normalization_2/beta', 'fc100/bias', 'batch_normalization_1/beta', 'conv2d_2/kernel', 'batch_normalization/moving_variance', 'batch_normalization_2/gamma', 'batch_normalization/moving_mean', 'batch_normalization_1/moving_variance', 'conv2d_1/bias', 'batch_normalization_2/moving_variance', 'batch_normalization/beta', 'batch_normalization_1/moving_mean', 'batch_normalization_2/moving_mean', 'fc100/kernel', 'batch_normalization_1/gamma', 'conv2d_2/bias', 'batch_normalization/gamma', 'conv2d/kernel']
2019-08-08 17:37:55,661 INFO  [src.meta.keras:221] (MainThread) conv output_names = ['fc100/Softmax', 'conv2d/kernel', 'conv2d/bias', 'batch_normalization/gamma', 'batch_normalization/beta', 'batch_normalization/moving_mean', 'batch_normalization/moving_variance', 'conv2d_1/kernel', 'conv2d_1/bias', 'batch_normalization_1/gamma', 'batch_normalization_1/beta', 'batch

W0808 17:38:03.801507 140100827197568 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


2019-08-08 17:38:03,802 INFO  [src.meta.keras:217] (MainThread) conv freeze_var_names = ['conv2d/bias', 'conv2d_1/kernel', 'batch_normalization_2/beta', 'fc100/bias', 'batch_normalization_1/beta', 'conv2d_2/kernel', 'batch_normalization/moving_variance', 'batch_normalization_2/gamma', 'batch_normalization/moving_mean', 'batch_normalization_1/moving_variance', 'conv2d_1/bias', 'batch_normalization_2/moving_variance', 'batch_normalization/beta', 'batch_normalization_1/moving_mean', 'batch_normalization_2/moving_mean', 'fc100/kernel', 'batch_normalization_1/gamma', 'conv2d_2/bias', 'batch_normalization/gamma', 'conv2d/kernel']
2019-08-08 17:38:03,803 INFO  [src.meta.keras:221] (MainThread) conv output_names = ['fc100/Softmax', 'conv2d/kernel', 'conv2d/bias', 'batch_normalization/gamma', 'batch_normalization/beta', 'batch_normalization/moving_mean', 'batch_normalization/moving_variance', 'conv2d_1/kernel', 'conv2d_1/bias', 'batch_normalization_1/gamma', 'batch_normalization_1/beta', 'batch

W0808 17:38:11.919552 140100827197568 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


2019-08-08 17:38:11,921 INFO  [src.meta.keras:217] (MainThread) conv freeze_var_names = ['conv2d/bias', 'conv2d_1/kernel', 'batch_normalization_2/beta', 'fc100/bias', 'batch_normalization_1/beta', 'conv2d_2/kernel', 'batch_normalization/moving_variance', 'batch_normalization_2/gamma', 'batch_normalization/moving_mean', 'batch_normalization_1/moving_variance', 'conv2d_1/bias', 'batch_normalization_2/moving_variance', 'batch_normalization/beta', 'batch_normalization_1/moving_mean', 'batch_normalization_2/moving_mean', 'fc100/kernel', 'batch_normalization_1/gamma', 'conv2d_2/bias', 'batch_normalization/gamma', 'conv2d/kernel']
2019-08-08 17:38:11,922 INFO  [src.meta.keras:221] (MainThread) conv output_names = ['fc100/Softmax', 'conv2d/kernel', 'conv2d/bias', 'batch_normalization/gamma', 'batch_normalization/beta', 'batch_normalization/moving_mean', 'batch_normalization/moving_variance', 'conv2d_1/kernel', 'conv2d_1/bias', 'batch_normalization_1/gamma', 'batch_normalization_1/beta', 'batch

In [11]:
m.epoch

11

In [12]:
m.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 75us/sample - loss: 2.9487 - acc: 0.2976


(0, [2.9487046257019043, 0.2976])

In [13]:
m.predict(test_images)

(0, array([[4.6424012e-07, 5.2555533e-06, 1.8432918e-03, ..., 8.7425396e-05,
         9.7608194e-04, 4.9521662e-03],
        [5.8053475e-04, 5.2950671e-04, 3.9766874e-04, ..., 3.1172194e-02,
         1.3336734e-04, 3.3685015e-04],
        [2.1916981e-07, 2.5617783e-05, 1.9058911e-04, ..., 6.3236867e-04,
         6.7025080e-04, 2.5662789e-04],
        ...,
        [8.6767471e-04, 9.6610643e-04, 1.1260538e-05, ..., 3.3607276e-03,
         2.0753562e-06, 2.4659007e-03],
        [9.6667281e-06, 4.5603465e-06, 2.2463712e-03, ..., 3.6006138e-02,
         1.3520994e-04, 7.7307485e-03],
        [8.8536966e-04, 9.9568488e-03, 2.4203486e-03, ..., 3.6148288e-07,
         2.6467685e-03, 2.2725342e-04]], dtype=float32))

In [14]:
m.save()

2019-08-08 17:38:14,120 DEBUG [src.meta.meta:96] (MainThread) conv Saving keras model to /space/code/deep-learning/models/conv_011/conv_011.h5...
2019-08-08 17:38:14,241 DEBUG [src.meta.meta:104] (MainThread) conv Saving keras model weights to /space/code/deep-learning/models/conv_011/conv_011_weights.h5...
2019-08-08 17:38:14,255 DEBUG [src.meta.meta:112] (MainThread) conv Saving keras model without optimizer to /space/code/deep-learning/models/conv_011/conv_011_no_optimizer.h5...


0

In [15]:
m.dump()

2019-08-08 17:38:14,285 DEBUG [src.utils.cuda_utils:72] (MainThread) Found 1 CUDA device.
2019-08-08 17:38:14,286 DEBUG [src.utils.cuda_utils:83] (MainThread) [GPU:0] Name: GeForce RTX 2080 Ti
2019-08-08 17:38:14,287 DEBUG [src.utils.cuda_utils:86] (MainThread) [GPU:0] Compute Capability: 7.5
2019-08-08 17:38:14,287 DEBUG [src.utils.cuda_utils:90] (MainThread) [GPU:0] Multiprocessors: 68
2019-08-08 17:38:14,288 DEBUG [src.utils.cuda_utils:91] (MainThread) [GPU:0] CUDA Cores: 13056
2019-08-08 17:38:14,288 DEBUG [src.utils.cuda_utils:95] (MainThread) [GPU:0] Concurrent threads: 69632
2019-08-08 17:38:14,289 DEBUG [src.utils.cuda_utils:99] (MainThread) [GPU:0] GPU clock: 1635 MHz
2019-08-08 17:38:14,290 DEBUG [src.utils.cuda_utils:103] (MainThread) [GPU:0] Memory clock: 7000 MHz
2019-08-08 17:38:14,371 ERROR [src.utils.cuda_utils:108] (MainThread) [GPU:0] cuCtxCreate failed with error code 2: out of memory
2019-08-08 17:38:14,371 DEBUG [src.meta.meta:123] (MainThread) conv mode = KERAS
20

0

In [16]:
m.save_to(TensorApi.TF_LITE, representative_data=train_images)

2019-08-08 17:38:14,406 DEBUG [src.meta.meta:222] (MainThread) conv Saving mode TF_LITE via mode KERAS...
2019-08-08 17:38:15,131 DEBUG [src.meta.meta:143] (MainThread) conv Deleting existing delegate...
2019-08-08 17:38:15,132 DEBUG [src.meta.keras:276] (MainThread) conv Loading keras model from /space/code/deep-learning/models/conv_011/conv_011.h5...
2019-08-08 17:38:16,020 INFO  [src.meta.meta:315] (MainThread) conv Converting to tflite INT8 model...
2019-08-08 17:38:17,796 INFO  [src.meta.meta:318] (MainThread) conv Saving tflite model to /space/code/deep-learning/models/conv_011/conv_011_int8.tflite...


0

In [17]:
m.save()

2019-08-08 17:38:17,805 DEBUG [src.meta.meta:96] (MainThread) conv Saving keras model to /space/code/deep-learning/models/conv_011/conv_011.h5...
2019-08-08 17:38:17,996 DEBUG [src.meta.meta:104] (MainThread) conv Saving keras model weights to /space/code/deep-learning/models/conv_011/conv_011_weights.h5...
2019-08-08 17:38:18,010 DEBUG [src.meta.meta:112] (MainThread) conv Saving keras model without optimizer to /space/code/deep-learning/models/conv_011/conv_011_no_optimizer.h5...


0

# Freeze

In [18]:
m.freeze_session()

2019-08-08 17:38:18,046 WARN  [src.meta.keras:190] (MainThread) conv Bug Workaround: Saving h5 file to ensure it can be loaded...
2019-08-08 17:38:18,081 WARN  [src.meta.keras:193] (MainThread) conv Bug Workaround: Saving h5 file without optimizer to ensure it can be loaded...
2019-08-08 17:38:18,097 WARN  [src.meta.keras:197] (MainThread) conv Bug Workaround: Clearing session...
2019-08-08 17:38:18,114 WARN  [src.meta.keras:201] (MainThread) conv Bug Workaround: Setting learning phase to 0...
2019-08-08 17:38:18,115 WARN  [src.meta.keras:204] (MainThread) conv Bug Workaround: Reloading keras model...


W0808 17:38:18.332382 140100827197568 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


2019-08-08 17:38:18,333 INFO  [src.meta.keras:217] (MainThread) conv freeze_var_names = ['conv2d/bias', 'conv2d_1/kernel', 'batch_normalization_2/beta', 'fc100/bias', 'batch_normalization_1/beta', 'conv2d_2/kernel', 'batch_normalization/moving_variance', 'batch_normalization_2/gamma', 'batch_normalization/moving_mean', 'batch_normalization_1/moving_variance', 'conv2d_1/bias', 'batch_normalization_2/moving_variance', 'batch_normalization/beta', 'batch_normalization_1/moving_mean', 'batch_normalization_2/moving_mean', 'fc100/kernel', 'batch_normalization_1/gamma', 'conv2d_2/bias', 'batch_normalization/gamma', 'conv2d/kernel']
2019-08-08 17:38:18,335 INFO  [src.meta.keras:221] (MainThread) conv output_names = ['fc100/Softmax', 'conv2d/kernel', 'conv2d/bias', 'batch_normalization/gamma', 'batch_normalization/beta', 'batch_normalization/moving_mean', 'batch_normalization/moving_variance', 'conv2d_1/kernel', 'conv2d_1/bias', 'batch_normalization_1/gamma', 'batch_normalization_1/beta', 'batch

0

In [19]:
m.save()

2019-08-08 17:38:19,139 DEBUG [src.meta.meta:96] (MainThread) conv Saving keras model to /space/code/deep-learning/models/conv_011/conv_011.h5...
2019-08-08 17:38:19,263 DEBUG [src.meta.meta:104] (MainThread) conv Saving keras model weights to /space/code/deep-learning/models/conv_011/conv_011_weights.h5...
2019-08-08 17:38:19,276 DEBUG [src.meta.meta:112] (MainThread) conv Saving keras model without optimizer to /space/code/deep-learning/models/conv_011/conv_011_no_optimizer.h5...


0

In [20]:
m.save_all(representative_data=train_images)

2019-08-08 17:38:19,307 DEBUG [src.meta.meta:96] (MainThread) conv Saving keras model to /space/code/deep-learning/models/conv_011/conv_011.h5...
2019-08-08 17:38:19,334 DEBUG [src.meta.meta:104] (MainThread) conv Saving keras model weights to /space/code/deep-learning/models/conv_011/conv_011_weights.h5...
2019-08-08 17:38:19,348 DEBUG [src.meta.meta:112] (MainThread) conv Saving keras model without optimizer to /space/code/deep-learning/models/conv_011/conv_011_no_optimizer.h5...
2019-08-08 17:38:19,377 DEBUG [src.meta.meta:222] (MainThread) conv Saving mode TENSOR_FLOW via mode KERAS...
2019-08-08 17:38:19,377 WARN  [src.meta.keras:190] (MainThread) conv Bug Workaround: Saving h5 file to ensure it can be loaded...
2019-08-08 17:38:19,403 WARN  [src.meta.keras:193] (MainThread) conv Bug Workaround: Saving h5 file without optimizer to ensure it can be loaded...
2019-08-08 17:38:19,422 WARN  [src.meta.keras:197] (MainThread) conv Bug Workaround: Clearing session...
2019-08-08 17:38:19,

W0808 17:38:19.650109 140100827197568 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


2019-08-08 17:38:19,651 INFO  [src.meta.keras:217] (MainThread) conv freeze_var_names = ['conv2d/bias', 'conv2d_1/kernel', 'batch_normalization_2/beta', 'fc100/bias', 'batch_normalization_1/beta', 'conv2d_2/kernel', 'batch_normalization/moving_variance', 'batch_normalization_2/gamma', 'batch_normalization/moving_mean', 'batch_normalization_1/moving_variance', 'conv2d_1/bias', 'batch_normalization_2/moving_variance', 'batch_normalization/beta', 'batch_normalization_1/moving_mean', 'batch_normalization_2/moving_mean', 'fc100/kernel', 'batch_normalization_1/gamma', 'conv2d_2/bias', 'batch_normalization/gamma', 'conv2d/kernel']
2019-08-08 17:38:19,652 INFO  [src.meta.keras:221] (MainThread) conv output_names = ['fc100/Softmax', 'conv2d/kernel', 'conv2d/bias', 'batch_normalization/gamma', 'batch_normalization/beta', 'batch_normalization/moving_mean', 'batch_normalization/moving_variance', 'conv2d_1/kernel', 'conv2d_1/bias', 'batch_normalization_1/gamma', 'batch_normalization_1/beta', 'batch

0

In [21]:
%%bash

find ../models -type f | sort | grep conv

../models/conv_001/conv_001_checkpoint.h5
../models/conv_001/conv_001.h5
../models/conv_001/conv_001_int8.tflite
../models/conv_001/conv_001_no_optimizer.h5
../models/conv_001/conv_001_weights.h5
../models/conv_002/conv_002_checkpoint.h5
../models/conv_002/conv_002.h5
../models/conv_002/conv_002_no_optimizer.h5
../models/conv_002/conv_002.pb
../models/conv_003/conv_003_checkpoint.h5
../models/conv_003/conv_003.h5
../models/conv_003/conv_003_int8.tflite
../models/conv_003/conv_003_no_optimizer.h5
../models/conv_003/conv_003.pb
../models/conv_003/conv_003_weights.h5
../models/conv_004/conv_004_checkpoint.h5
../models/conv_004/conv_004.h5
../models/conv_004/conv_004_no_optimizer.h5
../models/conv_004/conv_004.pb
../models/conv_005/conv_005_checkpoint.h5
../models/conv_005/conv_005.h5
../models/conv_005/conv_005_no_optimizer.h5
../models/conv_005/conv_005.pb
../models/conv_006/conv_006_checkpoint.h5
../models/conv_006/conv_006.h5
../models/conv_006/conv_006_no_optimizer.h5
../models/conv_0

In [22]:
if _DO_FIT:
    model.fit(train_images, train_labels, epochs=m.epoch+_ADDITIONAL_EPOCHS, initial_epoch=m.epoch)

2019-08-08 17:38:23,790 DEBUG [src.meta.keras:63] (MainThread) conv keras tensorboard callback attached. Visualize by running: 
2019-08-08 17:38:23,791 DEBUG [src.meta.keras:64] (MainThread) > tensorboard --logdir=/space/code/deep-learning/tmp/tensorboard/20190808-173821_conv
2019-08-08 17:38:23,792 DEBUG [src.meta.keras:66] (MainThread) conv keras checkpoint callback attached. Logging to /space/code/deep-learning/models/conv_{epoch:03d}/conv_{epoch:03d}_checkpoint.h5...
2019-08-08 17:38:23,793 INFO  [src.meta.keras:73] (MainThread) Creating /space/code/deep-learning/models/conv_012 (if not already exists)...
2019-08-08 17:38:23,794 INFO  [src.meta.keras:73] (MainThread) Creating /space/code/deep-learning/models/conv_013 (if not already exists)...
2019-08-08 17:38:23,795 INFO  [src.meta.keras:73] (MainThread) Creating /space/code/deep-learning/models/conv_014 (if not already exists)...
2019-08-08 17:38:23,795 INFO  [src.meta.keras:73] (MainThread) Creating /space/code/deep-learning/mod

In [23]:
m.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 82us/sample - loss: 2.9784 - acc: 0.3040


(0, [2.9784226978302004, 0.304])

In [24]:
m.predict(test_images)

(0, array([[9.92868718e-06, 3.12536977e-05, 9.01231449e-03, ...,
         1.46093778e-04, 2.97978427e-03, 6.92509161e-03],
        [2.81557022e-03, 2.24044570e-03, 2.89230444e-03, ...,
         2.16873474e-02, 1.26208016e-03, 2.02555908e-03],
        [2.40125996e-06, 1.05552375e-04, 5.70255506e-04, ...,
         4.00366727e-04, 4.26928792e-03, 6.80071942e-04],
        ...,
        [3.61690596e-02, 9.54690669e-03, 1.95732544e-04, ...,
         3.23741836e-03, 3.86979482e-05, 3.05418693e-03],
        [3.80086800e-04, 1.49936590e-04, 5.42628765e-02, ...,
         4.62245941e-02, 1.79184938e-03, 2.39915140e-02],
        [2.34272634e-03, 2.82269879e-03, 1.81688418e-04, ...,
         2.86076363e-09, 1.57820963e-04, 5.37552341e-06]], dtype=float32))

In [25]:
%%bash

find ../models -type f | sort | grep conv

../models/conv_001/conv_001_checkpoint.h5
../models/conv_001/conv_001.h5
../models/conv_001/conv_001_int8.tflite
../models/conv_001/conv_001_no_optimizer.h5
../models/conv_001/conv_001_weights.h5
../models/conv_002/conv_002_checkpoint.h5
../models/conv_002/conv_002.h5
../models/conv_002/conv_002_no_optimizer.h5
../models/conv_002/conv_002.pb
../models/conv_003/conv_003_checkpoint.h5
../models/conv_003/conv_003.h5
../models/conv_003/conv_003_int8.tflite
../models/conv_003/conv_003_no_optimizer.h5
../models/conv_003/conv_003.pb
../models/conv_003/conv_003_weights.h5
../models/conv_004/conv_004_checkpoint.h5
../models/conv_004/conv_004.h5
../models/conv_004/conv_004_no_optimizer.h5
../models/conv_004/conv_004.pb
../models/conv_005/conv_005_checkpoint.h5
../models/conv_005/conv_005.h5
../models/conv_005/conv_005_no_optimizer.h5
../models/conv_005/conv_005.pb
../models/conv_006/conv_006_checkpoint.h5
../models/conv_006/conv_006.h5
../models/conv_006/conv_006_no_optimizer.h5
../models/conv_0